In [1]:
import pandas as pd


In [2]:
movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'tmdb_5000_movies.csv'

## Data Analysis

In [ ]:
movies.head()

In [ ]:
movies.info()

In [ ]:
credits.head()

In [ ]:
credits.info()

In [ ]:
movies.merge(credits,on='title')

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies.head(1)

In [ ]:
movies.columns

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
# Atributes that will help in desiding tags
# genres
# id
# keywords
# title
# overview
# cast
# crew

In [ ]:
movies['original_language'].value_counts()
# most movies are of english language so this column has discarded

In [ ]:
movies=movies[['movie_id','title','genres','keywords','overview','cast','crew']]

In [ ]:
movies.head()

# Data Preprocessing

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.iloc[0].genres

In [ ]:
# convert this formet '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# to this ['Action','Adventure','Fantasy','Science Fiction']

In [ ]:
def convert(obj):
    L=[]
    for i in obj:
        L.append(i['name'])
    return L


In [ ]:
# convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [ ]:
movies['genres'] =  movies['genres'].apply(convert)

In [ ]:
movies

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies

In [ ]:
movies.isna().sum()

In [ ]:
movies['cast'][0]

In [ ]:
def convert2(obj):
    L = []
    counter =0
    for i in ast.literal_eval(obj):
        if counter != 3:
            counter+=1
            L.append(i['name'])
        else:
            break        
    return L   
        
    

In [ ]:
movies['cast'] = movies['cast'].apply(convert2)

In [ ]:
movies

In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L
        
        

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])


In [ ]:
movies

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv= CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.fit_transform(new_df['tags']).toarray().shape

In [ ]:
cv.get_feature_names()

In [ ]:
len(cv.get_feature_names())

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
ps.stem('dance')

In [ ]:
ps.stem('dancer')

In [ ]:
ps.stem('dancing')

In [ ]:
ps.stem('dancedfg')

In [ ]:
def stm(text):
    p=[]
    for i in text.split():
        p.append(ps.stem(i))
        
    return " ".join(p)
        
     

In [ ]:
new_df['tags']=new_df['tags'].apply(stm)

In [ ]:
new_df['tags'][0]

In [ ]:
ps.stem("attorney")

In [ ]:
#  'actions',
#  'activ',
#  'activist',
#  'activists',
#  'activities',
ps.stem('activities')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
cv.get_feature_names()

In [ ]:
vectors.shape

In [ ]:
# 1:21
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity.shape

In [ ]:
similarity[0]

In [ ]:
list(enumerate(similarity[0]))

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True)
# it does sorting on 1st column

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])
# giving list of most similar on above

In [ ]:
# Top 5 recommendation
sorted(list(enumerate(similarity[0])),reverse=True,key= lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True,key= lambda x:x[1])[1:6]    
#     movie_list = sorted(list(enumerate(distances)),reverse=True,key= lambda x:x[1])[1,6]
    
    for i in movie_list:
#         print(i[0])
        print(new_df.iloc[i[0]].title)

In [ ]:
new_df.iloc[65].title

In [ ]:
recommend("Batman Begins")

In [ ]:
new_df[new_df['title'] =='Batman Begins'].index[0]

In [ ]:
sorted(similarity[0],reverse=True)

In [ ]:
recommend('Avatar')

In [ ]:
recommend('John Carter')